In [1]:
import pandas as pd
import numpy as np
import time
import os
from IPython.display import display

In [2]:
import re 
def cleanup(text):

    import string
    punctuation = '[!\-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)    
    text_subbed = re.sub(pattern, ' ', text)
    text_subbed = re.sub("@", "", text_subbed)
    text_subbed = re.sub("http(|s):\/\/[^ \n]*", "", text_subbed)
    return text_subbed

In [3]:
treinamento1 = pd.read_excel("FelipeNeto_1.xlsx",sheet_name = 'Treinamento', index = False)
treinamento2 = pd.read_excel("FelipeNeto_2.xlsx",sheet_name = 'Treinamento', index = False)
treinamento3 = pd.read_excel("FelipeNeto_3.xlsx",sheet_name = 'Treinamento', index = False)
treinamento4 = pd.read_excel("FelipeNeto_4.xlsx",sheet_name = 'Treinamento', index = False)

lista_treinamentos=[treinamento1,treinamento2,treinamento3,treinamento4]

treinamento=pd.concat(lista_treinamentos,sort=False) #Juntando todas as planilhas

probabilidades=treinamento["Valor"].value_counts(True)#Calculando as frequências relativas de cada categoria

In [4]:
prob_ama = probabilidades[0]
prob_concorda = probabilidades[1]
prob_discorda = probabilidades[2]
prob_odeia = probabilidades[3]
prob_desrel = probabilidades[4]

In [5]:
ama = treinamento[treinamento["Valor"]==0]
ama = ama["Treinamento"].str.cat()

ama = cleanup(ama).split()
set(ama);
serie_ama=pd.Series(ama)

In [6]:
tabela_ama = serie_ama.value_counts(True)
tabela_ama;

In [7]:
concorda = treinamento[treinamento["Valor"]==1]
concorda = concorda["Treinamento"].str.cat()

concorda = cleanup(concorda).split()
set(concorda);
serie_concorda=pd.Series(concorda)

In [8]:
tabela_concorda = serie_concorda.value_counts(True)
tabela_concorda;

In [10]:
discorda = treinamento[treinamento["Valor"]==2]
discorda = discorda["Treinamento"].str.cat()

discorda = cleanup(discorda).split()
set(discorda);
serie_discorda=pd.Series(discorda)

In [11]:
tabela_discorda = serie_discorda.value_counts(True)
tabela_discorda;

In [12]:
odeia = treinamento[treinamento["Valor"]==3]
odeia = odeia["Treinamento"].str.cat()

odeia = cleanup(odeia).split()
set(odeia);

serie_odeia=pd.Series(odeia)

In [13]:
tabela_odeia = serie_odeia.value_counts(True)
tabela_odeia;

In [14]:
desrelacionado = treinamento[treinamento["Valor"]==4]
desrelacionado = desrelacionado["Treinamento"].str.cat()

desrelacionado = cleanup(desrelacionado).split()
set(desrelacionado);

serie_desrelacionado=pd.Series(desrelacionado)

In [15]:
tabela_desrel = serie_desrelacionado.value_counts(True)
tabela_desrel;

In [16]:
def classifica_palavra(frase):
    lista_palavras = frase.split()
    p0 = 1
    p1 = 1
    p2 = 1
    p3 = 1
    p4 = 1
    for word in lista_palavras:
        if word in tabela_ama.index:
            pword0 = tabela_ama[word]
        else:
            pword0 = 1
        if word in tabela_concorda.index:
            pword1 = tabela_concorda[word]
        else:
            pword1 = 1
        if word in tabela_discorda.index:
            pword2 = tabela_discorda[word]
        else:
            pword2 = 1
        if word in tabela_odeia.index:
            pword3 = tabela_odeia[word]
        else:
            pword3 = 1
        if word in tabela_desrel.index:
            pword4 = tabela_desrel[word]
        else:
            pword4 = 1
        p0 *= pword0
        p1 *= pword1
        p2 *= pword2
        p3 *= pword3
        p4 *= pword4
    if p0 == 1:
        p0 -= 1
    if p1 == 1:
        p1 -= 1
    if p2 == 1:
        p2 -= 1
    if p3 == 1:
        p3 -= 1
    if p4 == 1:
        p4 -= 1
    p0 *= prob_ama
    p1 *= prob_concorda
    p2 *= prob_discorda
    p3 *= prob_odeia
    p4 *= prob_desrel
    if (p0 > p1) and (p0 > p2) and (p0 > p3) and (p0 > p4):
        return "Ama"
    elif (p1 > p0) and (p1 > p2) and (p1 > p3) and (p1 > p4):
        return "Concorda"
    elif (p2 > p0) and (p2 > p1) and (p2 > p3) and (p2 > p4):
        return "Discorda"
    elif (p3 > p0) and (p3 > p1) and (p3 > p2) and (p3 > p4):
        return "Odeia"
    elif (p4 > p0) and (p4 > p1) and (p4 > p2) and (p4 > p3):
        return "Desrelacionado"
    
    return "p0: {0} - p1: {1} - p2: {2} - p3: {3} - p4: {4}".format(p0,p1,p2,p3,p4)

In [17]:
experimento = "herói"
print(classifica_palavra(experimento))

Concorda


In [123]:
i = 1
teste_x = pd.read_excel("FelipeNeto.xlsx",sheet_name = 'Teste', index = False)
while True:
    
    lista_teste =[teste_x]
    teste = pd.concat(lista_teste,sort=False) #Juntando todas as planilhas
    
    teste_idx = teste.set_index("Teste")
    lista_tweets = list(teste_idx.index)
    
    
    for tweet in lista_tweets:
        teste_idx.loc[tweet,"Classificação"] = classifica_palavra(tweet)
    
    writer = pd.ExcelWriter('FelipeNetoNaiveBayes_{0}.xlsx'.format(i), engine='xlsxwriter')
    teste_idx.to_excel(writer, sheet_name='Teste')
    
    writer.save()
    
    os.remove("FelipeNeto.xlsx")
    
    i += 1
    time.sleep(10)

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'FelipeNeto.xlsx'